# OCO2 - Count Cities and Plants inside a capture zone around the peak
Project for **Data For Good**, season 7. 

---

## Introduction


In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from numpy import exp, loadtxt, pi, sqrt
import math
import matplotlib
import matplotlib.pyplot as plt
import swiftclient
import json
from io import StringIO
import folium
from folium import plugins
import geopy
from shapely.geometry import Polygon
from geopy.distance import VincentyDistance

We establish the connexion with the swift server to pull and push ressources.

In [2]:
config_path = "../configs/config.json"

with open(config_path) as json_data_file:
    config = json.load(json_data_file)
    
def swift_con(config):
    user=config['swift_storage']['user']
    key=config['swift_storage']['key']
    auth_url=config['swift_storage']['auth_url']
    tenant_name=config['swift_storage']['tenant_name']
    auth_version=config['swift_storage']['auth_version']
    options = config['swift_storage']['options']
    return swiftclient.Connection(user=user,
                                  key=key,
                                  authurl=auth_url,
                                  os_options=options,
                                  tenant_name=tenant_name,
                                  auth_version=auth_version)

conn = swift_con(config)

## Retrieve Data

### Gaussian Peak Detection

The CSV contains the resulting detected peaks of the Gaussian method implemented in FC's paper, reproduced by Benoit. The CSV is stored on the Swift server.

We retrieve here the August 2018 data as an exemple.

In [3]:
from datetime import datetime
def to_date(a):
    return datetime.strptime(str(a), '%Y%m%d%H%M%S%f')

csv = conn.get_object("oco2", "/datasets/oco-2/peaks-detected/result_for_oco2_1501.csv")[1]
peak_fc = pd.read_csv(StringIO(str(csv, 'utf-8')), sep=";")
peak_fc['date'] = peak_fc['sounding_id'].apply(to_date)
peak_fc['sigma'] = peak_fc['sigma'].apply(abs)
peak_fc['amplitude'] = peak_fc['amplitude'].apply(abs)

peak_fc = gpd.GeoDataFrame(peak_fc, geometry=gpd.points_from_xy(peak_fc.longitude, peak_fc.latitude)).copy()
peak_fc.crs = {'init': 'epsg:4326'}

peak_fc.head()

,sounding_id,latitude,longitude,orbit,slope,intercept,amplitude,sigma,delta,R,windspeed_u,windspeed_v,surface_pressure,tcwv,gCO2_per_s,ktCO2_per_h,date,geometry
0,2015010102010436,-34.053768,177.050034,2662,-0.004189,396.374999,12.349512,27.090044,0.181865,0.705659,2.063626,-3.548534,1019.533020,34.468887,223652.620226,0.805149,2015-01-01 02:01:04.360,POINT (177.05003 -34.05377)
1,2015010102010905,-33.787537,176.996109,2662,-0.006491,396.376116,1.524073,10.516482,0.057816,0.623208,1.634062,-3.384151,1019.519958,34.735268,18405.576816,0.066260,2015-01-01 02:01:09.050,POINT (176.99611 -33.78754)
2,2015010102195372,28.690971,161.761734,2662,0.003902,398.971591,0.496041,16.246901,0.012180,0.503135,5.735664,1.822298,1015.385681,17.039511,46894.508017,0.168820,2015-01-01 02:19:53.720,POINT (161.76173 28.69097)
3,2015010103564534,22.098955,139.060425,2663,0.003469,398.327049,6.115482,16.643597,0.146586,0.500616,1.773756,-7.121686,1019.366699,16.780272,24071.441512,0.086657,2015-01-01 03:56:45.340,POINT (139.06042 22.09896)
4,2015010107153408,25.440878,88.655312,2665,-0.016556,399.036756,13.265101,10.515035,0.503280,0.709614,-1.282802,-0.865981,1014.103882,28.197447,307645.484122,1.107524,2015-01-01 07:15:34.080,POINT (88.65531 25.44088)


### Inventory Data

The CSV is stored on the GitHub.

In [35]:
year = "2019"

path_invent = "https://raw.githubusercontent.com/dataforgoodfr/batch7_satellite_ges/master/dataset/Output%20inventory%20data/Merge%20of%20peaks/CO2_emissions_peaks_merged_"+year+".csv"
invent = pd.read_csv(path_invent, sep=",", index_col=0)

invent = gpd.GeoDataFrame(invent, geometry=gpd.points_from_xy(invent.longitude, invent.latitude))
invent.crs = {'init': 'epsg:4326'}

invent = invent[invent['longitude'].notna()]
invent = invent[invent['latitude'].notna()]
invent = invent[invent['CO2/CO2e emissions (in tonnes per year)'].notna()]

invent_types = invent['CO2/CO2e emissions source'].unique()
print("Types of inventory: ")
print(invent_types)

invent.head()

Types of inventory: 
['City' 'Gas power plant' 'Oil power plant' 'Coal power plant'
 'Cement and Lime' 'Glass' 'Power and heat' 'Ceramics' 'Pulp and paper'
 'Chemicals' 'Iron and steel' 'Combustion' 'Non ferrous metals'
 'Mineral oil' 'Coke ovens']


,latitude,longitude,Data source,CO2/CO2e emissions source,CO2/CO2e emissions (in tonnes per year),CO2 or CO2e,geometry
0,43.653226,-79.383184,Opendatasoft,City,16151019.00,CO2,POINT (-79.38318 43.65323)
4,45.802578,9.086356,Opendatasoft,City,3728678.00,CO2,POINT (9.08636 45.80258)
5,37.668900,-122.080800,Opendatasoft,City,861854.00,CO2e,POINT (-122.08080 37.66890)
6,35.689634,139.692101,Opendatasoft,City,27611000.00,CO2e,POINT (139.69210 35.68963)
11,-10.249091,-48.324286,Opendatasoft,City,589055.31,CO2e,POINT (-48.32429 -10.24909)


## Inventory Capture

In [14]:
def get_direction_from_uv(u, v):
    ''' Retrieve the heading of a vector'''
    direction = 180/math.pi * math.atan2(u,v)+180
    return direction

def get_wind_norm_from_uv(u, v):
    ''' Retrieve the magitude of a vector'''
    return math.sqrt(pow(u,2)+pow(v,2))

def get_new_coord(lat, lon, d, b):
    ''' Calculate the arrival point of a vector, given a starting point, a distance and a direcion'''
    origin = geopy.Point(lat, lon)
    point = VincentyDistance(kilometers=d).destination(origin, b)
    return [point[1], point[0]]

def capture_zone(lat, lon, u, v, angle=50):
    ''' Calculates the capture zone around a point, given the point, a wind vector and a angme to shape the zone'''
    wind_heading = get_direction_from_uv(u, v)
    wind_norm = 3 * np.log(get_wind_norm_from_uv(u,v)*3.6 +1)+1

    # BACK LINE
    # 1st point (back - 6h wind)
    point_1 = get_new_coord(lat, lon, wind_norm*6, wind_heading+180)
    # 2nd point (back - 6h wind - 50°)
    point_2 = get_new_coord(lat, lon, wind_norm*6, wind_heading+180-angle)
    # 3rd point (back - 6h wind - 50°)
    point_3 = get_new_coord(lat, lon, wind_norm*6, wind_heading+180+angle)

    # FRONT LINE
    # 4th point (front - 24h wind - 20°)
    point_4 = get_new_coord(lat, lon, wind_norm*24, wind_heading+angle)
    # 5th point (front - 24h wind - 20°)
    point_5 = get_new_coord(lat, lon, wind_norm*24, wind_heading-angle)
    # 6th point (front - 24h wind)
    point_6 = get_new_coord(lat, lon, wind_norm*24, wind_heading)

    points = [point_1, point_2, point_4, point_6, point_5, point_3, point_1]
    return points

def capture_df(row):
    ''' Apply the capture zone function to a dataset row'''
    return Polygon(capture_zone(row["latitude"], row["longitude"], row['windspeed_v'], row['windspeed_u']))

def join_and_count_one(peaks, invent, title):
    ''' Spacially join 3 datasets (Polygon, Point and Point)'''
    #intersect cities
    peaks_intersect_invent  = gpd.sjoin(peaks, invent.loc[:, ['CO2/CO2e emissions (in tonnes per year)','geometry']], how='left', op='intersects').rename(columns={"index_right": "index_"+str(title)})
    peaks_intersect_ag = peaks_intersect_invent.groupby([peaks_intersect_invent.index, 'sounding_id'])["index_"+str(title)].apply(list).reset_index()
    peaks_intersect_ag["number_"+str(title)] = peaks_intersect_ag["index_"+str(title)].apply(lambda x: np.count_nonzero(~np.isnan(x)))
    peaks_meters_invent = peaks.merge(peaks_intersect_ag, how='left', on='sounding_id')
    
    peaks_meters_invent = peaks_meters_invent.drop(columns=['level_0'])
    return peaks_meters_invent

### Gaussian-only Peaks

We set a capture zone and spatially join the data for FC's peaks:

In [15]:
# Remove far east and west to avoid side effects on captures zones 
peaks_fc = peak_fc.loc[(peak_fc["longitude"] < 170) & (peak_fc["longitude"] > -170), :]
peaks_fc['geometry'] = peaks_fc.apply(capture_df, axis=1)

for cl in invent_types:
    invent_pt = invent[invent['CO2/CO2e emissions source'] == cl]
    peaks_fc = join_and_count_one(peaks_fc, invent_pt, cl)
    
peaks_fc.head(50)

<ipython-input-9-d6e42feeafde>:13: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  point = VincentyDistance(kilometers=d).destination(origin, b)
<ipython-input-15-a6aa5c53ae3e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  peaks_fc['geometry'] = peaks_fc.apply(capture_df, axis=1)


,sounding_id,latitude,longitude,orbit,slope,intercept,amplitude,sigma,delta,R,...,index_Iron and steel,number_Iron and steel,index_Combustion,number_Combustion,index_Non ferrous metals,number_Non ferrous metals,index_Mineral oil,number_Mineral oil,index_Coke ovens,number_Coke ovens
0,2015010102195372,28.690971,161.761734,2662,0.003902,398.971591,0.496041,16.246901,0.012180,0.503135,...,[nan],0,[nan],0,[nan],0,[nan],0,[nan],0
1,2015010103564534,22.098955,139.060425,2663,0.003469,398.327049,6.115482,16.643597,0.146586,0.500616,...,[nan],0,[nan],0,[nan],0,[nan],0,[nan],0
2,2015010107153408,25.440878,88.655312,2665,-0.016556,399.036756,13.265101,10.515035,0.503280,0.709614,...,[nan],0,[nan],0,[nan],0,[nan],0,[nan],0
3,2015010108302501,-54.517262,84.449547,2666,0.005714,394.863990,4.267378,18.559233,0.091730,0.547798,...,[nan],0,[nan],0,[nan],0,[nan],0,[nan],0
4,2015010108490474,7.789284,68.618759,2666,-0.002647,397.640725,2.401853,6.527914,0.146785,0.517912,...,[nan],0,[nan],0,[nan],0,[nan],0,[nan],0
5,2015010108490576,7.823544,68.615707,2666,-0.002223,397.651355,0.666586,5.782463,0.045989,0.529903,...,[nan],0,[nan],0,[nan],0,[nan],0,[nan],0
6,2015010108550305,27.417038,63.311405,2666,-0.003320,399.297440,0.052845,10.922466,0.001930,0.540308,...,[nan],0,[nan],0,[nan],0,[nan],0,[nan],0
7,2015010110171871,-27.944864,52.021336,2667,-0.004023,397.092901,19.233105,22.329799,0.343617,0.567947,...,[nan],0,[nan],0,[nan],0,[nan],0,[nan],0
8,2015010110171974,-27.913832,51.982479,2667,-0.004717,397.089630,19.573490,28.891999,0.270272,0.624511,...,[nan],0,[nan],0,[nan],0,[nan],0,[nan],0
9,2015010110172072,-27.841391,51.986526,2667,-0.005112,397.072088,18.862900,29.325258,0.256612,0.635424,...,[nan],0,[nan],0,[nan],0,[nan],0,[nan],0


In [16]:
peaks_fc.describe()

,sounding_id,latitude,longitude,orbit,slope,intercept,amplitude,sigma,delta,R,...,number_Glass,number_Power and heat,number_Ceramics,number_Pulp and paper,number_Chemicals,number_Iron and steel,number_Combustion,number_Non ferrous metals,number_Mineral oil,number_Coke ovens
count,1.413000e+03,1413.000000,1413.000000,1413.000000,1413.000000,1413.000000,1413.000000,1413.000000,1413.000000,1413.000000,...,1413.000000,1413.000000,1413.000000,1413.000000,1413.000000,1413.000000,1413.000000,1413.0,1413.0,1413.0
mean,2.015011e+15,-3.112578,3.250107,2838.135881,-0.000032,397.787980,14.571054,16.753032,0.359457,0.593115,...,0.000708,0.007077,0.008493,0.004954,0.002123,0.002123,0.016985,0.0,0.0,0.0
std,1.028778e+09,22.772906,88.002740,149.974511,0.005941,1.301799,15.359279,6.739570,0.367820,0.075024,...,0.026603,0.266029,0.319235,0.186220,0.079809,0.079809,0.638470,0.0,0.0,0.0
min,2.015010e+15,-58.314800,-166.061554,2662.000000,-0.025444,394.637652,0.002130,2.218839,0.000075,0.500051,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
25%,2.015010e+15,-23.801931,-82.809250,2710.000000,-0.004610,396.871431,4.825394,11.438821,0.126645,0.531469,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
50%,2.015011e+15,-1.753876,10.332716,2815.000000,0.000650,397.424321,10.840705,15.621290,0.258672,0.577349,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
75%,2.015012e+15,16.099979,69.530357,2887.000000,0.004562,398.669471,19.294509,21.310058,0.454254,0.634205,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
max,2.015013e+15,42.820824,167.942184,3112.000000,0.016324,402.002674,139.032147,33.322151,2.939649,0.865666,...,1.000000,10.000000,12.000000,7.000000,3.000000,3.000000,24.000000,0.0,0.0,0.0


## CSV save

In [64]:
for i in range (2001, 2013):
    csv = conn.get_object("oco2", "/datasets/oco-2/peaks-detected/result_for_oco2_"+str(i)+".csv")[1]
    peak_fc = pd.read_csv(StringIO(str(csv, 'utf-8')), sep=";")
    peak_fc['date'] = peak_fc['sounding_id'].apply(to_date)
    peak_fc['sigma'] = peak_fc['sigma'].apply(abs)
    peak_fc['amplitude'] = peak_fc['amplitude'].apply(abs)
    peak_fc = gpd.GeoDataFrame(peak_fc, geometry=gpd.points_from_xy(peak_fc.longitude, peak_fc.latitude)).copy()
    peak_fc.crs = {'init': 'epsg:4326'}
    
    peaks_fc = peak_fc.loc[(peak_fc["longitude"] < 170) & (peak_fc["longitude"] > -170), :]
    peaks_fc['geometry'] = peaks_fc.apply(capture_df, axis=1)

    for cl in invent_types:
        invent_pt = invent[invent['CO2/CO2e emissions source'] == cl]
        peaks_fc = join_and_count_one(peaks_fc, invent_pt, cl)
        
    peaks_fc.to_csv(r'../dataset/peaks_and_invent/peaks_and_invent_'+str(i)+'.csv', index = False)


<ipython-input-9-d6e42feeafde>:13: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  point = VincentyDistance(kilometers=d).destination(origin, b)
<ipython-input-64-91e66fbaf9b6>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  peaks_fc['geometry'] = peaks_fc.apply(capture_df, axis=1)
<ipython-input-9-d6e42feeafde>:13: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  point = VincentyDistance(kilometers=d).destination(origin,

ClientException: Object GET failed: https://storage.gra.cloud.ovh.net/v1/AUTH_2aaacef8e88a4ca897bb93b984bd04dd/oco2//datasets/oco-2/peaks-detected/result_for_oco2_2005.csv 404 Not Found  [first 60 chars of response] b'<html><h1>Not Found</h1><p>The resource could not be found.<'

In [30]:
path_test = "../dataset/peaks_and_invent/peaks_and_invent_1903.csv"
test = pd.read_csv(path_test, sep=",", index_col=0)
test.head()

,latitude,longitude,orbit,slope,intercept,amplitude,sigma,delta,R,windspeed_u,...,index_Iron and steel,number_Iron and steel,index_Combustion,number_Combustion,index_Non ferrous metals,number_Non ferrous metals,index_Mineral oil,number_Mineral oil,index_Coke ovens,number_Coke ovens
sounding_id,,,,,,,,,,,,,,,,,,,,,
2019030100271608,-16.832216,-162.028778,24796,-0.004824,406.686617,5.425395,8.730678,0.247910,0.535715,-4.954284,...,[nan],0,[nan],0,[nan],0,[nan],0,[nan],0
2019030100271678,-16.794907,-162.036835,24796,-0.005197,406.693058,2.095986,9.137989,0.091506,0.523415,-4.996274,...,[nan],0,[nan],0,[nan],0,[nan],0,[nan],0
2019030102142903,11.252090,167.095169,24797,0.009478,408.488802,8.960968,8.758122,0.408182,0.572906,-9.526621,...,[nan],0,[nan],0,[nan],0,[nan],0,[nan],0
2019030102143208,11.367613,167.069168,24797,0.006009,408.614616,10.909251,9.944954,0.437625,0.619813,-9.591630,...,[nan],0,[nan],0,[nan],0,[nan],0,[nan],0
2019030102143131,11.402847,167.058075,24797,0.006472,408.582778,17.281968,14.315919,0.481597,0.657686,-9.611622,...,[nan],0,[nan],0,[nan],0,[nan],0,[nan],0


In [19]:
#export
def peaks_capture_map(peaks, invent, mapbox_token = None, month="1908"):
    """
    Create map with peaks (marker + capture zone) and inventory
    :param peaks: GeoDataFrame, Dataframe containing the peaks we want to display.
    :param plants: GeoDataFrame, Dataframe containing all registered plants.
    :param plants_coal: GeoDataFrame, Dataframe containing all registered coal plants.
    :param cities: GeoDataFrame, Dataframe containing all registered big cities.
    :return:
    """
    # Initialize Map
    peaks_capture = folium.Map([40, -0], zoom_start=3, tiles=None)
    folium.TileLayer("CartoDB dark_matter", name="Dark mode").add_to(peaks_capture)
    folium.TileLayer("OpenStreetMap", name="Open Street Map").add_to(peaks_capture)
    if mapbox_token is not None:
        folium.TileLayer(tiles="Mapbox", name="Satellite", API_key=mapbox_token).add_to(peaks_capture)
    
    
    # Adding detected peaks
    peaks_group = folium.FeatureGroup(name="Peaks").add_to(peaks_capture)
    peaks_group_capture = folium.FeatureGroup(name=" - 50km Capture Zone", show=True).add_to(peaks_capture)
    for index, row in peaks.iterrows():
        radius = row["amplitude"]/20
        tooltip =  "["+str(round(row['latitude'],2))+" ; "+str(round(row['longitude'],2))+"]"
        color="#FF3333" # red
        sounding = str(row['sounding_id'])
        date = str(row['date'])
        orbit = str(row['orbit'])  
        
        popup_html="""<h4>"""+tooltip+"""</h4>"""+date+"""<p>sounding_id: """+sounding+"""</br>orbit: """+orbit+"""</p>"""
        popup_html+='<p><input type="button" value="Show plot"'
        # Injecting JavaScript in popup to fire the Dash Callback
        popup_html+='onclick="\
            let bco_input = parent.document.getElementById(\'input_sounding\'); \
            let lastValue = bco_input.value;'
        popup_html+=f'bco_input.value = \'{sounding}\';'
        popup_html+="let bco_event = new Event('input', { bubbles: true });\
            bco_event.simulated = true;\
            let tracker = bco_input._valueTracker;\
            if (tracker) {\
            tracker.setValue(lastValue);\
            }\
            bco_input.dispatchEvent(bco_event);\
            \"/></p>"
        
        popup=folium.Popup(popup_html, max_width=450)
        
        peaks_group_capture.add_child(folium.GeoJson(row['geometry'], name=" - Capture Zone"))
        
        peaks_group.add_child(folium.CircleMarker(location=(row["latitude"],
                                      row["longitude"]),
                            radius=radius,
                            color=color,
                            tooltip=sounding,
                            popup=popup,
                            fill=True))

    d={}
    invent_types = invent['CO2/CO2e emissions source'].unique()
    inventory = folium.FeatureGroup(name="Inventory").add_to(peaks_capture)
    for cl in invent_types:
        d["{0}".format(cl)]=folium.plugins.FeatureGroupSubGroup(inventory, name=" - "+cl).add_to(peaks_capture)
        
    for index, row in invent.iterrows():
        radius = 1
        color="#368534" # green

        tooltip =  "["+str(round(row['latitude'],2))+" ; "+str(round(row['longitude'],2))+"]"
        pop = str(round(row['CO2/CO2e emissions (in tonnes per year)'],0))
        title = "" + '{:,.2f}'.format(round(row['CO2/CO2e emissions (in tonnes per year)']/1000,0)) + " kilo-ton of "+ row['CO2 or CO2e'] + " per year"
        subtitle= "~ " + '{:,.2f}'.format(round(row['CO2/CO2e emissions (in tonnes per year)']/(365*24),0)) + " ton of CO2 per hour (est.)"
        popup_html = """<h4>"""+title+"""</h4><p><h5>("""+ subtitle + """) </h5></p><p>GPS: """+tooltip+"""</p>""" + str(row['CO2/CO2e emissions source']) + ", from " + str(row['Data source']) + """</p>"""
        popup=folium.Popup(popup_html, max_width=450)

        d[row['CO2/CO2e emissions source']].add_child(folium.CircleMarker(location=(row["latitude"],
                                      row["longitude"]),
                            radius=radius,
                            color=color,
                            tooltip= str(row['CO2/CO2e emissions source']),
                            popup=popup,
                            fill=True))

    peaks_capture.keep_in_front(peaks_group)
    
    folium.map.LayerControl(collapsed=False).add_to(peaks_capture)
    
    plugins.Fullscreen(
        position='topleft',
        title='Expand me',
        title_cancel='Exit me',
        force_separate_button=True
    ).add_to(peaks_capture)

    minimap = plugins.MiniMap(position='bottomleft')
    peaks_capture.add_child(minimap)
    
    print("Saving "+month+ " peak map...")
    peaks_capture.save("../front/peak_maps/peaks_capture_map_"+month+".html")
    return "Done "+month+ " peak map..."

In [36]:
for i in range (2001, 2007):
    csv = conn.get_object("oco2", "/datasets/oco-2/peaks-detected/result_for_oco2_"+str(i)+".csv")[1]
    peak_fc = pd.read_csv(StringIO(str(csv, 'utf-8')), sep=";")
    peak_fc = peak_fc[peak_fc.delta > 1]
    peak_fc['date'] = peak_fc['sounding_id'].apply(to_date)
    peak_fc['sigma'] = peak_fc['sigma'].apply(abs)
    peak_fc['amplitude'] = peak_fc['amplitude'].apply(abs)
    peak_fc = gpd.GeoDataFrame(peak_fc, geometry=gpd.points_from_xy(peak_fc.longitude, peak_fc.latitude)).copy()
    peak_fc.crs = {'init': 'epsg:4326'}
    
    peaks_fc = peak_fc.loc[(peak_fc["longitude"] < 170) & (peak_fc["longitude"] > -170), :]
    peaks_fc['geometry'] = peaks_fc.apply(capture_df, axis=1)

    for cl in invent_types:
        invent_pt = invent[invent['CO2/CO2e emissions source'] == cl]
        peaks_fc = join_and_count_one(peaks_fc, invent_pt, cl)
        
    peaks_fc.to_csv(r'../dataset/peaks_and_invent/peaks_and_invent_'+str(i)+'.csv', index = False)
    
    peaks_capture_map(peaks_fc, invent, mapbox_token = None, month=str(i))


<ipython-input-14-87c65b893d19>:13: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  point = VincentyDistance(kilometers=d).destination(origin, b)


Saving 2001 peak map...


<ipython-input-14-87c65b893d19>:13: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  point = VincentyDistance(kilometers=d).destination(origin, b)
<ipython-input-36-08c7999808f7>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  peaks_fc['geometry'] = peaks_fc.apply(capture_df, axis=1)


Saving 2002 peak map...


<ipython-input-14-87c65b893d19>:13: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  point = VincentyDistance(kilometers=d).destination(origin, b)


Saving 2003 peak map...


<ipython-input-14-87c65b893d19>:13: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  point = VincentyDistance(kilometers=d).destination(origin, b)


Saving 2004 peak map...


ClientException: Object GET failed: https://storage.gra.cloud.ovh.net/v1/AUTH_2aaacef8e88a4ca897bb93b984bd04dd/oco2//datasets/oco-2/peaks-detected/result_for_oco2_2005.csv 404 Not Found  [first 60 chars of response] b'<html><h1>Not Found</h1><p>The resource could not be found.<'